In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00


In [2]:
import time
import requests
from datasets import load_dataset
from tqdm.notebook import tqdm

import sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef

import json
import re

In [3]:
class YandexGPTApi:
    """
    API for accessing GigaChat
    List of available models: https://cloud.yandex.com/en/docs/yandexgpt/concepts/models#
    Requires Catalog Indicator and API key. Instruction to get: https://habr.com/ru/articles/780008/
    Timeout is used with 'yandexgpt' model as requests for it are asynchronious
    """

    def __init__(
        self,
        api_key: str,
        catalog_indicator: str,
        model: str = "yandexgpt-lite",
        temperature: float = 0.6,
        max_tokens: int = 2000,
        timeout: int = 3,
        system_prompt: str = "Ты учитель русского языка и помогаешь своим ученикам."
    ):
        self.api_key = api_key
        self.catalog_indicator = catalog_indicator
        self.model = model
        self.temperature = temperature
        self.max_tokens = str(max_tokens)
        self.timeout = timeout
        self.system_prompt = system_prompt

    def prompt(self, text: str) -> str:

        if self.model == "yandexgpt-lite":
            completion = "completion"
        else:
            completion = "completionAsync"

        if self.system_prompt != "":
            messages = [
                  {"role": "system", "text": self.system_prompt},
                  {"role": "user", "text": text}]
        else:
            messages = [{"role": "user", "text": text}]
        prompt = {
            "modelUri": f"gpt://{self.catalog_indicator}/yandexgpt",
            "completionOptions": {
                "stream": False,
                "temperature": 0.6,
                "maxTokens": "2000",
            },
            "messages": messages,
        }

        url = "https://llm.api.cloud.yandex.net/foundationModels/v1/" + completion

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Api-Key {self.api_key}",
        }

        response = requests.post(url, headers=headers, json=prompt)

        if self.model == "yandexgpt-lite":
            timeout = time.time() + 60 * self.timeout
            return response.json()["result"]["alternatives"][0]["message"]["text"]
        if self.model == "yandexgpt":
            id = response.json()["id"]
            response = requests.get(
                f"https://llm.api.cloud.yandex.net/operations/{id}", headers=headers
            )
            timeout = time.time() + 60 * self.timeout
            while response.json()["done"] != True:
                response = requests.get(
                    f"https://llm.api.cloud.yandex.net/operations/{id}", headers=headers
                )
                if time.time() > timeout:
                    break
            return response.json()["response"]["alternatives"][0]["message"]["text"]

    def switch_system_prompt(self, system_prompt: str) -> None:
        self.system_prompt = system_prompt

In [4]:
data = load_dataset('RussianNLP/rucola', split = 'validation')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7869 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2787 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2789 [00:00<?, ? examples/s]

In [5]:
yandex_gpt = YandexGPTApi(catalog_indicator='b1gd5vs65ojp5em83q5n', api_key='AQVN28TRiNtVgaorCchGxPQCx37hRo8WE270ctRh', model="yandexgpt", temperature = 0.5, timeout=3) # ТУТ СКРОЙ ПОТОМ КЛЮЧИ
yandex_gpt.switch_system_prompt("Ты учитель русского языка. Твоя задача — помогать своим ученикам.")
#yandex_gpt.prompt("Правильно ли следующее предложение с точки зрения грамматики русского языка: У президента почти полмиллиона стихотворений забыли.")

In [ ]:
bad_sentences = [11, 54, 57, 108, 329, 335, 358, 609, 772, 1009, 1043, 1057, 1069, 1119, 1127, 1209, 1218, 1251, 1305, 1313,
                 1367, 1514, 1527, 1557, 1573, 1586, 1688, 1893, 1939,
                 2070, 2081, 2102, 2120, 2179, 2181, 2259,
                 2342, 2352, 2427, 2437, 2570, 2592, 2614, 2620, 2630, 2642, 2705, 2744, 2752, 2771]
bad_sentences.extend([355, 1110, 1139, 1174, 1185, 1585, 1630, 1644, 1652, 1742, 1797,
1842, 1873, 2027, 2171, 2307, 2329, 2404, 453, 595, 1093, 1252, 1935, 2206, 2245, 2272, 2283, 2288, 2714])

### label

In [ ]:
i = 0
for message in tqdm(data['sentence']):
  if i in bad_sentences:
    pass
  else:
    a = yandex_gpt.prompt(f'Правильно ли следующее предложение с точки зрения грамматики русского языка: "{message}"? Ответь только одно слово: да или нет.')
    if a.lower().startswith('да'):
      with open('yagpt_rucola_ru_11_05_2024_label_withrole_2.tsv', 'a') as f:
        f.write(str(i)+'\t'+'1'+'\n')
    else:
      with open('yagpt_rucola_ru_11_05_2024_label_withrole_2.tsv', 'a') as f:
        f.write(str(i)+'\t'+'0'+'\n')
  i += 1

  0%|          | 0/1002 [00:00<?, ?it/s]

### label, category

In [9]:
i = 0
for message in tqdm(data['sentence']):
  if i in bad_sentences:
    pass
  else:
    a = yandex_gpt.prompt(f'''Есть ли ошибка в предложении: "{message}"?
    Если ошибки нет, ответь только 1 слово "нет" (пожалуйста, не добавляй ничего), если ошибка есть – обязательно укажи также тип ошибки: синтаксис, морфология, семантика или галлюцинации.
    Примеры ошибочных предложений: морфология ― "Увидал, что у санитара нет сапогов ― велел дать.",
    синтаксис ― "Этим летом не никуда ездили.", семантика ― "Он живет в небольшом городке у Волги.",
    галлюцинации ― "Люк останавливает удачу от этого.". Пожалуйста, внимательно проверь предложение.''')
    #print(a)
    if a.lower().startswith('нет') or (re.findall('нет .*ошибки|ошибки .*нет|нет .*ошибок|ошибок .*нет', a.lower()) != []
                                      or (re.findall('отсутству(е|ю)т', a.lower())) != []):
      with open('/content/yandexgpt_rucola_ru_07_05_2024_label+category_2.tsv', 'a') as f:
        f.write(str(i)+'\t'+'0' + '\n')
    elif 'морфолог' in a.lower():
      with open('/content/yandexgpt_rucola_ru_07_05_2024_label+category_2.tsv', 'a') as f:
        f.write(str(i)+'\t'+'Morphology' + '\n')
    elif 'синтакси' in a.lower():
      with open('/content/yandexgpt_rucola_ru_07_05_2024_label+category_2.tsv', 'a') as f:
        f.write(str(i)+'\t'+'Syntax' + '\n')
    elif 'семанти' in a.lower():
      with open('/content/yandexgpt_rucola_ru_07_05_2024_label+category_2.tsv', 'a') as f:
        f.write(str(i)+'\t'+'Semantics' + '\n')
    elif 'галлюцина' in a.lower():
      with open('/content/yandexgpt_rucola_ru_07_05_2024_label+category_2.tsv', 'a') as f:
        f.write(str(i)+'\t'+'Hallucination' + '\n')
    else:
      with open('/content/yandexgpt_rucola_ru_07_05_2024_label+category_2.tsv', 'a') as f:
        f.write(str(i)+'\t'+a.upper() + '\n')
  i += 1

  0%|          | 0/1 [00:00<?, ?it/s]

### label+explanation

In [12]:
for j in tqdm(range(len(data['sentence']))):
    #print(j)
    if j in bad_sentences:
      pass
    else:
      a = yandex_gpt.prompt(f'''Перед тобой предложение на русском языке, в котором, возможно, есть ошибка: {data['sentence'][j]}
Ответь, есть ли ошибка в этом предложении. Если предложение неправильное, объясни, в чем ошибка. Объясни ошибку подробно, пожалуйста.'''
)
      a = a.replace('\n', ' ')
      if a.lower().startswith('нет') or (re.findall('нет .*ошибки|ошибки .*нет|нет .*ошибок|ошибок .*нет', a.lower()) != []
      or (re.findall('ошибк.*отсутству(е|ю)т', a.lower())) != []) or (re.findall('отсутству(е|ю)т.*ошибк', a.lower())) != []:
        label = '1'
      else:
        label = '0'
      #print(j, a)
      with open('/content/yandexgpt_rucola_ru_09_05_2024_label+explanation_2.tsv', 'a') as f:
        f.write(str(j) + '\t' + label + '\t' + data['sentence'][j] + '\t' + a + '\n')

  0%|          | 0/1 [00:00<?, ?it/s]

### label+correction

In [19]:
for j in tqdm(range(len(data['sentence']))): # И ТУТ
    #print(j)
    if j in bad_sentences:
      pass
    else:
      a = yandex_gpt.prompt(f'''Перед тобой предложение на русском языке, в котором, возможно, есть ошибка: {data['sentence'][j]}
      Если в предложении есть ошибка, исправь предложение и напиши правильный вариант.
      Если предложение не содержит ошибок, ответь "в предложении нет ошибок".
      Пожалуйста, проверяй предложение очень внимательно.
      '''
)
      a = a.replace('\n', ' ')
      if a.lower().startswith('нет') or (re.findall('нет .*ошибки|ошибки .*нет|нет .*ошибок|ошибок .*нет', a.lower()) != []
      or (re.findall('ошибк.*отсутству(е|ю)т', a.lower())) != []) or (re.findall('отсутству(е|ю)т.*ошибк', a.lower())) != []:
        label = '1'
      else:
        label = '0'
      #print(j, a)
      if label == '0':
        with open('yandexgpt_rucola_ru_06_05_2024_label+correction_3.tsv', 'a') as f:
          f.write(str(j) + '\t'  + data['sentence'][j] + '\t' + label + '\t' + a + '\n')
      else:
        with open('yandexgpt_rucola_ru_06_05_2024_label+correction_3.tsv', 'a') as f:
          f.write(str(j) + '\t'  + data['sentence'][j] + '\t' + label + '\t' + 'OK' + '\n')

  0%|          | 0/1 [00:00<?, ?it/s]

### label+category+explanation

In [22]:
for j in tqdm(range(len(data['sentence']))): # И ТУТ
    #print(j)
    if j in bad_sentences:
      pass
    else:
      a = yandex_gpt.prompt(f'''
Перед тобой предложение на русском языке, в котором, возможно, есть ошибка: {data['sentence'][j]}
Если в предложении есть ошибка, ответь, к какой категории она относится, и объясни, в чем заключается ошибка.
Ошибка может относиться к одной из следующих категорий: морфология, синтаксис, семантика или галлюцинации.
В ответе может быть только одна категория ошибки, нельзя приводить несколько. Примеры ошибочных предложений:
синтаксис ― "Этим летом не никуда ездили.", семантика ― "Он живет в небольшом городке у Волги.",
морфология ― "Увидал, что у санитара нет сапогов ― велел дать.",
галлюцинации ― "Люк останавливает удачу от этого.". Напиши ответ в формате: категория ошибки:, объяснение: [твое объяснение].
Если предложение не содержит ошибок, ответь "в предложении нет ошибок".
Пожалуйста, проверь предложение очень внимательно, это очень важно.
      '''
)
      a = a.replace('\n', ' ')
      #print(a)
      if (a.lower().startswith('нет') or re.findall('нет .*ошибки|ошибки .*нет|нет .*ошибок|ошибок .*нет', a.lower()) != []
      or re.findall('ошибк.*отсутству(е|ю)т', a.lower()) != [] or (re.findall('отсутству(е|ю)т.*ошибк', a.lower())) != [] or
      'ошибки нет' in a.lower() or 'отсутству' in a.lower() or 'не обнаружен' in a.lower() or 'нет ошибки' in a.lower()):
        label = '1'
      else:
        label = '0'
        if 'морфологи' in a.lower():
          category = 'Morphology'
        elif 'синтакси' in a.lower():
          category = 'Syntax'
        elif 'семанти' in a.lower():
          category = 'Semantics'
        elif 'галлюцина' in a.lower():
          category = 'Hallucination'
        else:
          category = 'ERROR'
      #print(j, a)

      if label == '0':
        with open('yandexgpt_rucola_ru_13_05_2024_label+cat+expl.tsv', 'a') as f:
          f.write(str(j) + '\t'  + data['sentence'][j] + '\t' + label + '\t' + category + '\t' + a + '\n')
      else:
        with open('yandexgpt_rucola_ru_13_05_2024_label+cat+expl.tsv', 'a') as f:
          f.write(str(j) + '\t'  + data['sentence'][j] + '\t' + label + '\t' + '0' + '\t' + 'OK' + '\n')

  0%|          | 0/1 [00:00<?, ?it/s]

### label+category+correction

In [26]:
for j in tqdm(range(len(data['sentence']))): # И ТУТ
    #print(j)
    if j in bad_sentences:
      pass
    else:
      a = yandex_gpt.prompt(f'''Перед тобой предложение на русском языке, в котором, возможно, есть ошибка: {data['sentence'][j]}
Если в предложении есть ошибка, ответь, к какой категории она относится, и напиши исправленное предложение. Не объясняй свое решение.
Ошибка может относиться к одной из следующих категорий: морфологическая, синтаксическая, семантическая или галлюцинации.
Примеры ошибочных предложений:
синтаксическая ошибка ― "Этим летом не никуда ездили.", семантическая ошибка ― "Он живет в небольшом городке у Волги.",
морфологическая ошибка ― "Увидал, что у санитара нет сапогов ― велел дать.",
галлюцинация ― "Люк останавливает удачу от этого.". Напиши ответ в формате: категория ошибки: [синтаксическая/семантическая/морфологическая/галлюцинация],
исправленное предложение: [правильное предложение].
Если предложение не содержит ошибок, ответь только "в предложении нет ошибок".
Пожалуйста, проверь предложение очень внимательно, это очень важно.'''
)
      a = a.replace('\n', ' ')
      #print(a)
      if (re.findall('нет .*(ошибок|ошибки)', a.lower()) != [] or re.findall('(ошибок|ошибки) нет', a.lower()) != []
or (re.findall('ошибк(а|и) отсутству(е|ю)т', a.lower()) != []) or 'не обнаружен' in a.lower()):
        label = '1'
      else:
        label = '0'
        if 'морфологи' in a.lower():
          category = 'Morphology'
        elif 'синтакси' in a.lower():
          category = 'Syntax'
        elif 'семанти' in a.lower():
          category = 'Semantics'
        elif 'галлюцина' in a.lower():
          category = 'Hallucination'
        else:
          category = 'ERROR'
      #print(j, a)


      if label == '0':
        with open('yandexgpt_rucola_ru_16_05_2024_label+cat+corr.tsv', 'a') as f:
          f.write(str(j) + '\t'  + data['sentence'][j] + '\t' + label + '\t' + category + '\t' + a + '\n')
      else:
        with open('yandexgpt_rucola_ru_16_05_2024_label+cat+corr.tsv', 'a') as f:
          f.write(str(j) + '\t'  + data['sentence'][j] + '\t' + label + '\t' + '0' + '\t' + 'OK' + '\n')

  0%|          | 0/1 [00:00<?, ?it/s]

### label+explanation+correction

In [29]:
for j in tqdm(range(len(data['sentence']))): # И ТУТ
    #print(j)
    if j in bad_sentences:
      pass
    else:
      a = yandex_gpt.prompt(f'''
Перед тобой предложение на русском языке, в котором, возможно, есть ошибка: {data['sentence'][j]}
Если в предложении есть ошибка, объясни, в чем именно ошибка, и исправь предложение.
Если ты нашел ошибку, напиши ответ в формате: объяснение: [твое объяснение], исправленное предложение: [правильное предложение].
Если предложение не содержит ошибок, ответь только "в предложении нет ошибок".
Пожалуйста, если ты нашел ошибку, опиши ее максимально подробно, чтобы помочь своему ученику.
'''
)
      a = a.replace('\n', ' ')
      #print(a)
      if (re.findall('нет .*(ошибок|ошибки)', a.lower()) != [] or re.findall('(ошибок|ошибки) нет', a.lower()) != []
or (re.findall('ошибк(а|и) отсутству(е|ю)т', a.lower()) != []) or 'не обнаружен' in a.lower()):
        label = '1'
      else:
        label = '0'


      if label == '0':
        expl = a[a.find('бъяснение:')+11:a.find('справленное предложение:')-2]
        corr = a[a.find('справленное предложение:')+25:]
        with open('yandexgpt_rucola_ru_18_05_2024_label+expl+corr.tsv', 'a') as f:
          f.write(str(j) + '\t'  + data['sentence'][j] + '\t' + label +  '\t' + expl + '\t' +corr + '\n')
      else:
        with open('yandexgpt_rucola_ru_18_05_2024_label+expl+corr.tsv', 'a') as f:
          f.write(str(j) + '\t'  + data['sentence'][j] + '\t' + label + '\t' + 'OK' + '\t' + 'OK' + '\n')

  0%|          | 0/1 [00:00<?, ?it/s]

### label+category+explanation+correction

In [31]:
for j in tqdm(range(len(data['sentence']))): # И ТУТ
    #print(j)
    if j in bad_sentences:
      pass
    else:
      a = yandex_gpt.prompt(f'''
Перед тобой предложение на русском языке, в котором, возможно, есть ошибка: {data['sentence'][j]}
Если в предложении есть ошибка, ответь, к какой категории она относится. Также объясни, в чем именно ошибка, и исправь предложение.
Если предложение не содержит ошибок, ответь только "в предложении нет ошибок".
Если ты нашел ошибку, напиши ответ в формате: категория: [категория ошибки], объяснение: [твое объяснение], исправленное предложение: [правильное предложение].
Ошибка может относиться к одной из следующих категорий: морфологическая, синтаксическая, семантическая или галлюцинации.
Примеры ошибочных предложений: синтаксическая ошибка ― "Этим летом не никуда ездили.", семантическая ошибка ― "Он живет в небольшом городке у Волги.",
морфологическая ошибка ― "Увидал, что у санитара нет сапогов ― велел дать.", галлюцинаци ― "Люк останавливает удачу от этого.".
Пожалуйста, если ты нашел ошибку, опиши ее максимально подробно, чтобы помочь своему ученику.
'''
)
      a = a.replace('\n', ' ')
      #print(a)
      if (re.findall('нет .*(ошибок|ошибки)', a.lower()) != [] or re.findall('(ошибок|ошибки) нет', a.lower()) != []
or (re.findall('ошибк(а|и) отсутству(е|ю)т', a.lower()) != []) or 'не обнаружен' in a.lower()):
        label = '1'
      else:
        label = '0'


      if label == '0':
        cat = a[a.find('атегория:')+10:a.find('бъяснение:')-2]
        #expl = a[a.find('бъяснение:')+11:a.find('справленное предложение:')-2]
        corr = a[a.find('справленное предложение:')+25:]
        with open('yandexgpt_rucola_ru_21_05_2024_label+cat+expl+corr_2.tsv', 'a') as f:
          f.write(str(j) + '\t'  + data['sentence'][j] + '\t' + label +  '\t' + cat + '\t' + a + '\t' +corr + '\n')
      else:
        with open('yandexgpt_rucola_ru_21_05_2024_label+cat+expl+corr_2.tsv', 'a') as f:
          f.write(str(j) + '\t'  + data['sentence'][j] + '\t' + label + '\t' + '0' + '\t' + 'OK' + '\t' + 'OK' + '\n')

  0%|          | 0/1 [00:00<?, ?it/s]